<a href="https://colab.research.google.com/github/Wally0924/Embedded-Systems/blob/main/homework/transportation/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

## dataset

In [ ]:
# TODO upload your dataset or use roboflow download code
!curl -L {download_url} > roboflow.zip
!unzip ./roboflow.zip

### train model

In [ ]:
!pip install ultralytics

In [ ]:
# TODO train the model with your dataset
# recommend at least train 50 epochs
!yolo detect train data={yaml_path} model=yolov8n.pt epochs=10 imgsz=640

#### test model

In [ ]:
from ultralytics import YOLO
# TODO test your model with test img in dataset
model = YOLO({best_model_path})
model.predict({test_img}, save=True, imgsz=640, conf=0.5)

## export yolov8 to trt

In [ ]:
from pathlib import Path
# TODO your train dir ex. runs/detect/train/
train_path = Path("")
model_path = train_path / "weights"
pt_file = str(model_path / "best.pt")
onnx_file = str(model_path / "best.onnx")
trt_file = str(model_path / "best.engine")

In [ ]:
# tensorRT engine
!yolo export model={pt_file} format=onnx

In [ ]:
# download onnx
from google.colab import files
files.download(onnx_file)